<a href="https://colab.research.google.com/github/anandusateeshNSUT/News-Headline-Classification/blob/main/News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [6]:
df=pd.read_csv('/content/gdrive/MyDrive/NLP/datasets/Preprocessed.csv')
print(df.head())

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  million americans roll sleeves omicrontargeted...  U.S. NEWS   
1  american airlines flyer charged banned life pu...  U.S. NEWS   
2                funniest tweets cats dogs week sept     COMEDY   
3                  funniest tweets parents week sept  PARENTING   
4  woman called cops black birdwatcher loses laws...  U.S. NEWS   

                                   short_description               authors  \
0  health experts said early predict whether dema...  Carla K. Johnson, AP   
1  subdued passengers crew fled back aircraft con...        Mary Papenfuss   
2                    dog d

In [7]:
category_to_int = {'POLITICS': 0, 'BUSINESS': 1, 'SPORTS': 2, 'ENTERTAINMENT': 3, 'TRAVEL': 4}
df['label'] = df.category.map(category_to_int).fillna(5).astype(int)

In [8]:
df = df.drop(['link', 'short_description', 'authors', 'date'], axis=1)

In [9]:
num_classes = len(df["category"].value_counts())

In [10]:
df.head(100)

,headline,category,label
0,million americans roll sleeves omicrontargeted...,U.S. NEWS,5
1,american airlines flyer charged banned life pu...,U.S. NEWS,5
2,funniest tweets cats dogs week sept,COMEDY,5
3,funniest tweets parents week sept,PARENTING,5
4,woman called cops black birdwatcher loses laws...,U.S. NEWS,5
...,...,...,...
95,fastmoving fairview fire kills least california,U.S. NEWS,5
96,kody clemens strikes mvp shohei ohtani trails ...,SPORTS,2
97,mississippi governor says water pressure solid...,U.S. NEWS,5
98,meta parent company instagram fined million ir...,U.S. NEWS,5


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.groupby(['category', 'label', 'data_type']).count()


headline
category       label data_type          
ARTS           5     train          1286
                     val             222
ARTS & CULTURE 5     train          1120
                     val             219
BLACK VOICES   5     train          3859
...                                  ...
WOMEN          5     val             564
WORLD NEWS     5     train          2814
                     val             485
WORLDPOST      5     train          2178
                     val             400

[84 rows x 1 columns]

In [12]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [13]:
df.head()

,headline,category,label,data_type
0,million americans roll sleeves omicrontargeted...,U.S. NEWS,5,train
1,american airlines flyer charged banned life pu...,U.S. NEWS,5,train
2,funniest tweets cats dogs week sept,COMEDY,5,val
3,funniest tweets parents week sept,PARENTING,5,train
4,woman called cops black birdwatcher loses laws...,U.S. NEWS,5,train


In [14]:
df = df.dropna(subset=['headline'])

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].headline.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].headline.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)



input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32




dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup


optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 1


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in category_to_int.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in category_to_int.values():
        if label not in label_dict_inverse:
            continue
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [20]:
import os
directory = 'News_Classification'
if not os.path.exists(directory):
    os.makedirs(directory)

In [21]:
import random
import numpy as np
seed_val = 16
random.seed(seed_val)
np.random.seed(seed_val)

torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)

from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
         
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad(), autocast():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
      
    model.train()
    
    loss_train_total = 0
 
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        with autocast():
            outputs = model(**inputs)
            loss = outputs[0]
            
        loss_train_total += loss.item()
        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'News_Classification/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/5564 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.6410559809824959
Validation loss: 0.5124776711167726
F1 Score (Weighted): 0.8047863063340485


In [22]:


model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)


model.load_state_dict(torch.load('News_Classification/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: POLITICS
Accuracy: 4057/5339

Class: BUSINESS
Accuracy: 217/898

Class: SPORTS
Accuracy: 499/762

Class: ENTERTAINMENT
Accuracy: 1643/2604

Class: TRAVEL
Accuracy: 1067/1485



In [23]:

model.save_pretrained('News_Classification/finetuned_BERT_epoch_1')

In [27]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoConfig
import torch.nn.functional as F

category_map=['POLITICS', 'BUSINESS', 'SPORTS', 'ENTERTAINMENT', 'TRAVEL']

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model_config = AutoConfig.from_pretrained('/content/News_Classification/finetuned_BERT_epoch_1/config.json')


model = AutoModelForSequenceClassification.from_pretrained('/content/News_Classification/finetuned_BERT_epoch_1/pytorch_model.bin', config=model_config)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    inputs.to(device)
    with torch.no_grad():
        logits = model(**inputs)[0]
        preds = F.softmax(logits, dim=1)
    return preds.detach().cpu().numpy()
test_text = ["U.S.-China tensions escalate as Biden administration reaffirms commitment to Taiwan's security", "Serena Williams announces retirement from professional tennis, leaving a lasting legacy on the sport"]
predictions = [predict(text) for text in test_text]


for text, pred in zip(test_text, predictions):
    label_index = np.argmax(pred)
    category_name = category_map[label_index]
    print(f'Text: {text}\nPredicted category: {category_name}\n')


Text: U.S.-China tensions escalate as Biden administration reaffirms commitment to Taiwan's security
Predicted category: POLITICS

Text: Serena Williams announces retirement from professional tennis, leaving a lasting legacy on the sport
Predicted category: SPORTS

